In [12]:
import pymupdf
import os
import json
import sys
import time
import concurrent.futures
from pathlib import Path

In [13]:
# number of threads
num_threads = os.cpu_count()
num_threads

# path
p = Path('/lus/eagle/projects/argonne_tpc/siebenschuh/aurora_gpt/joint')
pdf_files = list(p.rglob("*.pdf"))

In [7]:
time.time()

1726698882.1864052

In [8]:
# truncate list
pdf_files = pdf_files[:50]

In [14]:
def extract_text_from_pdf(pdf_file):
    try:
        # open the PDF
        doc = pymupdf.open(pdf_file)
        
        # Extract text from all pages
        text = ""
        for page_num in range(doc.page_count):
            page = doc.load_page(page_num)
            text += page.get_text()

        # Return the file path and extracted text as a tuple
        return str(pdf_file), text
    except Exception as e:
        print(f"Error processing {pdf_file}: {e}")
        return str(pdf_file), ""

# Parallelized version of your code
def parallel_parse_pdfs(pdf_files, num_processes=4):
    pdf_text_dict = {}

    # Use ProcessPoolExecutor to parallelize the parsing
    with concurrent.futures.ProcessPoolExecutor(max_workers=num_processes) as executor:
        # Submit tasks and collect results
        results = list(executor.map(extract_text_from_pdf, pdf_files))

    # Store results in the dictionary
    pdf_text_dict = dict(results)
    
    return pdf_text_dict

In [ ]:
%%time 
time_dict = {}
for num_processes in [1, 4, 8, 16]: # [1,2,4,6,8,10,12,14,16,20,32,50,64]:
    t0 = time.time()
    # Run the parallelized parsing function
    pdf_text_dict = parallel_parse_pdfs(pdf_files, num_processes)
    t1 = time.time()
    # append
    time_dict = {'duration' : t1 - t0, 'num_processes' : num_processes}
    
    # store locally
    # = = = = = = = = = 
    # sort the dictionary by its keys (file paths) alphabetically
    sorted_pdf_text_dict = dict(sorted(pdf_text_dict.items()))
    # store the sorted dictionary to a file
    output_path = Path(f'./tmp/sorted_pdf_num_processes_{num_processes}.json')
    with open(output_path, 'w') as json_file:
        json.dump(sorted_pdf_text_dict, json_file)
    
    print(f"Sorted dictionary saved to: {output_path}")

MuPDF error: syntax error: could not parse color space (254 0 R)

MuPDF error: syntax error: could not parse color space (525 0 R)

MuPDF error: syntax error: could not parse color space (215 0 R)

MuPDF error: syntax error: cannot find ExtGState resource 'GS10'

MuPDF error: unsupported error: cannot create appearance stream for Screen annotations

MuPDF error: unsupported error: cannot create appearance stream for Screen annotations

MuPDF error: unsupported error: cannot create appearance stream for Screen annotations

MuPDF error: unsupported error: cannot create appearance stream for Screen annotations

MuPDF error: unsupported error: cannot create appearance stream for Screen annotations

MuPDF error: unsupported error: cannot create appearance stream for Screen annotations

MuPDF error: unsupported error: cannot create appearance stream for Screen annotations

MuPDF error: unsupported error: cannot create appearance stream for Screen annotations

MuPDF error: syntax error: could

In [ ]:
len(pdf_files)

In [ ]:
time_dict

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.bar(time_dict['num_processes'], time_dict['duration'])

# Add labels and title
plt.xlabel('Number of Processes')
plt.ylabel('Duration (seconds)')
plt.title('Duration vs. Number of Processes')

# Display the plot
plt.show()